<a href="https://colab.research.google.com/github/ahmadf20/text-mining-naive-bayes/blob/main/DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation


In [1]:
import re
import json
from collections import Counter

# sklearn - machine learning library
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

# numpy - scientific computing library
import numpy as np

# pandas - python data analysis library
import pandas as pd

# sastrawi - stemming library (bahasa indonesia)
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# tqdm - progress bar library
from tqdm import tqdm

     |████████████████████████████████| 215kB 5.6MB/s 


Data diperoleh dari https://ptik.fkip.uns.ac.id/skripsi/home/judul

In [2]:
!wget 'https://github.com/ahmadf20/text-mining-naive-bayes/raw/main/raw_datasets.xlsx'
!wget 'https://github.com/ahmadf20/text-mining-naive-bayes/raw/main/stopwords-id.json'

--2020-12-30 20:39:12--  https://github.com/ahmadf20/text-mining-naive-bayes/raw/main/raw_datasets.xlsx
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ahmadf20/text-mining-naive-bayes/main/raw_datasets.xlsx [following]
--2020-12-30 20:39:12--  https://raw.githubusercontent.com/ahmadf20/text-mining-naive-bayes/main/raw_datasets.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26955 (26K) [application/octet-stream]
Saving to: ‘raw_datasets.xlsx’

raw_datasets.xlsx   100%[===================>]  26.32K  --.-KB/s    in 0.001s  

2020-12-30 20:39:13 (20.3 MB/s) - ‘raw_datasets.xlsx’ saved [26955/26955

In [3]:
df = pd.read_excel("raw_datasets.xlsx")
df.head(10)

,NIM/Nama,Judul,Jenis,Pembimbing,Status
0,K3515028 / HERMAN SANJAYA,UPAYA MENINGKATKAN HASIL BELAJAR DAN KEAKTIFAN...,Penelitian Tindakan Kelas,"Dwi Maryono, S.Si, M.Kom",Revisi
1,NaN,NaN,NaN,"Dr. Agus Efendi, M.Pd",NaN
2,K3515018 / Ely Rohmawati,UPAYA MENINGKATKAN HASIL BELAJAR DAN KEAKTIFAN...,Penelitian Tindakan Kelas,"Basori, S.Pd., M.Pd",Acc Ujian Skripsi
3,NaN,NaN,NaN,"Puspanda Hatta, S. Kom., M.Eng",NaN
4,K3517018 / Dimas Galang Ramadhan,TINJAUAN PENGARUH GAME-BASED LEARNING TERHADAP...,Penelitian Kualitatif,"Cucuk Wawan Budiyanto, ST., Ph.D",Mendapat Dosen Pembimbing
5,NaN,NaN,NaN,"Rosihan Ari Yuana, S.Si, M.Kom",NaN
6,K3514037 / Putri Wulandari,Studi Evaluasi terhadap Game Edukatif untuk Pe...,Penelitian Kualitatif,"Endar Suprih W., S.T.,M.Eng",Acc Ujian Skripsi
7,NaN,NaN,NaN,"Rosihan Ari Yuana, S.Si, M.Kom",NaN
8,K3516024 / Febri Candra Nur Baiti,Strategi Practice Rehearsal Pair dalam model p...,Penelitian Kuantitatif,"Basori, S.Pd., M.Pd",Acc Seminar Proposal
9,NaN,NaN,NaN,"Cucuk Wawan Budiyanto, ST., Ph.D",NaN


In [4]:
df['Pembimbing2'] = np.zeros
for index, row in df.iterrows():
  if(index%2 != 0):
    df.iloc[index-1]['Pembimbing2'] = row['Pembimbing']

df = df.loc[::2]


In [5]:
df.columns

Index(['NIM/Nama', 'Judul', 'Jenis', 'Pembimbing', 'Status', 'Pembimbing2'], dtype='object')

In [6]:
df = df[['NIM/Nama', 'Judul', 'Jenis', 'Pembimbing', 'Pembimbing2','Status']]

In [7]:
df

,NIM/Nama,Judul,Jenis,Pembimbing,Pembimbing2,Status
0,K3515028 / HERMAN SANJAYA,UPAYA MENINGKATKAN HASIL BELAJAR DAN KEAKTIFAN...,Penelitian Tindakan Kelas,"Dwi Maryono, S.Si, M.Kom","Dr. Agus Efendi, M.Pd",Revisi
2,K3515018 / Ely Rohmawati,UPAYA MENINGKATKAN HASIL BELAJAR DAN KEAKTIFAN...,Penelitian Tindakan Kelas,"Basori, S.Pd., M.Pd","Puspanda Hatta, S. Kom., M.Eng",Acc Ujian Skripsi
4,K3517018 / Dimas Galang Ramadhan,TINJAUAN PENGARUH GAME-BASED LEARNING TERHADAP...,Penelitian Kualitatif,"Cucuk Wawan Budiyanto, ST., Ph.D","Rosihan Ari Yuana, S.Si, M.Kom",Mendapat Dosen Pembimbing
6,K3514037 / Putri Wulandari,Studi Evaluasi terhadap Game Edukatif untuk Pe...,Penelitian Kualitatif,"Endar Suprih W., S.T.,M.Eng","Rosihan Ari Yuana, S.Si, M.Kom",Acc Ujian Skripsi
8,K3516024 / Febri Candra Nur Baiti,Strategi Practice Rehearsal Pair dalam model p...,Penelitian Kuantitatif,"Basori, S.Pd., M.Pd","Cucuk Wawan Budiyanto, ST., Ph.D",Acc Seminar Proposal
...,...,...,...,...,...,...
338,K3515050 / Rizka Rizkianna,Analisis Efektivitas Penggunaan Teknologi Asis...,Mixed Method,"Dr. Agus Efendi, M.Pd","Dwi Maryono, S.Si, M.Kom",Pembimbingan Seminar
340,K3517039 / Muhammad Luthfi Zulfikar,Analisis dan Evaluasi Penerapan Sistem Informa...,Research and Development,"Nurcahya Pradana Taufik Prakisya, S. Kom, M. CS.","Yusfia Hafid Aristyagama, S. T, M. T.",Mendapat Dosen Pembimbing
342,K3516021 / Fadilla Shinta Devi,Analisis dan Evaluasi Aplikasi Berbasis Mobile...,Penelitian Kualitatif,"Puspanda Hatta, S. Kom., M.Eng","Rosihan Ari Yuana, S.Si, M.Kom",Acc Ujian Skripsi
344,K3516019 / Dimas Maenaka Aji Perdana,Analisis dan Evaluasi Aplikasi Belajar Pemrogr...,Penelitian Kualitatif,"Rosihan Ari Yuana, S.Si, M.Kom","Yusfia Hafid Aristyagama, S. T, M. T.",Acc Seminar Proposal


In [8]:
df.to_excel('dataset_pre.xlsx', index=False)

# Data Processing
Membersihkan dokumen dengan cara:

1.   Merubah kapilal menjadi nonkapital (Case Folding)  
2.   Memotong kalimat menjadi kumpulan kata (Tokenizing)
3.   Menghilangkan kata yang tidak penting (Filtering/Stopword)
4.   Mengubah kata ke bentuk dasar (Stemming)

In [9]:
# import file yang berisi kata-kata tidak penting
data_stopword = json.load(open('stopwords-id.json','r'))

# menjadikan array stopword menjadi unordered collection (set())
# agar dapat dilakukan operasi matematis seperti union, intersection, symmetric difference
stopword = set(data_stopword)

# method untuk cleaning dokumen
def clean(doc):
    # merubah kapital menjadi non kapital & menghilangkan kata tidak penting
    stop_free = " ".join([i for i in doc.lower().split() if i not in stopword])
    # mengubah kata ke bentuk dasar
    stemmer = StemmerFactory().create_stemmer()
    normalized = stemmer.stem(stop_free)
    # memotong kalimat menjadi kumpulan kata
    y = normalized.split()
    return y

# method untuk cleaning dokumen berupa array
def clean_with_loop(arr):
    hasil = []
    progress = tqdm(arr)
    for item in progress:
        progress.set_description("Membersihkan dokumen")
        cleaned = clean(item)
        cleaned = ' '.join(cleaned)
        hasil.append(cleaned)
    return hasil

In [10]:
# import dataset 
dataset = pd.read_excel('dataset_pre.xlsx')
print(dataset.head())

# mengambil hanya kolom Teks dan disimpan di variabel skripsi (yang siap dibersihkan)
skripsi = []
for index, row in dataset.iterrows():
    skripsi.append(row["Judul"])

print("Jumlah skripsi: ", len(skripsi))

# mengambil hanya kolom label dalam variabel y_train
y_train = []
for index, row in dataset.iterrows():
    y_train.append(row["Pembimbing"])

print("Jumlah Pembimbing: ", len(y_train))

# membersihkan dokumen skripsi
skripsi_bersih = clean_with_loop(skripsi)

Membersihkan dokumen:   0%|          | 0/174 [00:00<?, ?it/s]

                            NIM/Nama  ...                     Status
0          K3515028 / HERMAN SANJAYA  ...                     Revisi
1           K3515018 / Ely Rohmawati  ...          Acc Ujian Skripsi
2   K3517018 / Dimas Galang Ramadhan  ...  Mendapat Dosen Pembimbing
3         K3514037 / Putri Wulandari  ...          Acc Ujian Skripsi
4  K3516024 / Febri Candra Nur Baiti  ...       Acc Seminar Proposal

[5 rows x 6 columns]
Jumlah skripsi:  174
Jumlah Pembimbing:  174


Membersihkan dokumen: 100%|██████████| 174/174 [01:44<00:00,  1.66it/s]


In [11]:
skripsi_bersih

['upaya tingkat hasil ajar aktif siswa model ajar blended learning dengan bantu edmodo ajar teknologi layan jaring kelas xi tkj 2 smk negeri 1 banyudono',
 'upaya tingkat hasil ajar aktif siswa dengan model ajar discovery learning brainstorming ajar jaring dasar kelas x tkj 1 smk negeri 1 banyudono',
 'tinjau pengaruh game-based learning kembang computational thinking skills perspektif teori brennan resnick',
 'studi evaluasi game edukatif didik sd kelas rendah ajar hitung tinjau aspek gamifikasi kurikulum',
 'strategi practice rehearsal pair model ajar active learning kreativitas collaboration skill siswa ajar dasar desain grafis',
 'sistem dukung putus klasterisasi publikasi ilmiah k-means fkip uns',
 'sistem ajar adaptif bas api pemrograman java dengan dekat model siswa instruksional',
 'simulasi ukur kerja otomatisasi jaring laboratorium komputer studi laboratorium komputer ptik fkip uns',
 'scoping review guna augmented reality media ajar tingkat motivasi siswa',
 'relevansi kulia

# Membentuk TF-IDF


In [12]:
# pembentukan vektor tf-idf untuk pembobotan kata
vectorizer = TfidfVectorizer(stop_words=data_stopword)
x_train = vectorizer.fit_transform(skripsi_bersih)
print(x_train)

  (0, 63)	0.277889561826779
  (0, 369)	0.15918334851967542
  (0, 522)	0.13387055931543607
  (0, 578)	0.2356055554924108
  (0, 624)	0.25736377079184025
  (0, 271)	0.17501429083825534
  (0, 258)	0.1821994006756096
  (0, 313)	0.30905141967871436
  (0, 561)	0.2356055554924108
  (0, 157)	0.33378597776457564
  (0, 62)	0.24203285525848484
  (0, 131)	0.15918334851967542
  (0, 314)	0.1480612000372425
  (0, 76)	0.20693395876147086
  (0, 353)	0.13865755748473663
  (0, 514)	0.12048878486857281
  (0, 21)	0.22979135562317057
  (0, 20)	0.24903192784677672
  (0, 211)	0.1510375428236743
  (0, 575)	0.16277279120294108
  (0, 598)	0.2915019714302074
  (1, 122)	0.18694445419398253
  (1, 80)	0.3745763104991978
  (1, 142)	0.3271249849763996
  (1, 63)	0.311849070150965
  :	:
  (171, 65)	0.16960135941099092
  (171, 47)	0.3371789989072145
  (171, 170)	0.22723350040348184
  (171, 576)	0.23104858119022206
  (171, 20)	0.19203606231851025
  (172, 31)	0.2752981193532517
  (172, 38)	0.30901749664166284
  (172, 403)	0

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['kali', 'kurangnya', 'lamanya', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [13]:
totalDosen = set(y_train)
totalDosen

{'Aris Budianto, S.T., M.Eng',
 'Basori, S.Pd., M.Pd',
 'Cucuk Wawan Budiyanto, ST., Ph.D',
 'Dr. Agus Efendi, M.Pd',
 'Dwi Maryono, S.Si, M.Kom',
 'Endar Suprih W., S.T.,M.Eng',
 'Febri Liantoni, S. ST., M. Kom',
 'Nurcahya Pradana Taufik Prakisya, S. Kom, M. CS.',
 'Puspanda Hatta, S. Kom., M.Eng',
 'Rosihan Ari Yuana, S.Si, M.Kom'}

In [14]:
# pengelompokan dokumen dengan dengan naive bayes
modelNB = MultinomialNB()
modelNB.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Pengujian


In [15]:
kalimat_tes = ["UPAYA MENINGKATKAN HASIL BELAJAR DAN KEAKTIFAN SISWA MELALUI MODEL PEMBELAJARAN BLENDED LEARNING DENGAN BERBANTUKAN EDMODO PADA MATA PELAJARAN TEKNOLOGI LAYANAN JARINGAN KELAS XI TKJ 2 DI SMK NEGERI 1 BANYUDONO",
               "UPAYA MENINGKATKAN HASIL BELAJAR DAN KEAKTIFAN SISWA DENGAN MODEL PEMBELAJARAN DISCOVERY LEARNING BRAINSTORMING DALAM MATA PELAJARAN JARINGAN DASAR KELAS X TKJ 1 DI SMK NEGERI 1 BANYUDONO",
               "TINJAUAN PENGARUH GAME-BASED LEARNING TERHADAP PERKEMBANGAN COMPUTATIONAL THINKING SKILLS DARI PERSPEKTIF TEORI BRENNAN & RESNICK",
               "Studi Evaluasi terhadap Game Edukatif untuk Pendidikan SD (Kelas Rendah) dalam Pembelajaran Berhitung ditinjau dari Aspek Gamifikasi dan Kurikulum"]

# membersihkan dokumen pengujian
kalimat_tes_bersih = clean_with_loop(kalimat_tes)

# loop untuk prediksi kelompok
for teks in kalimat_tes_bersih:
    arr_teks = []
    arr_teks.append(teks)
    vektor = vectorizer.transform(arr_teks)
    prediksi_label_knn = modelNB.predict(vektor)
    print("\n- Judul :"+teks, ":\n" + "Pembimbing : " + prediksi_label_knn[0])

Membersihkan dokumen: 100%|██████████| 4/4 [00:03<00:00,  1.27it/s]


- Judul :upaya tingkat hasil ajar aktif siswa model ajar blended learning dengan bantu edmodo ajar teknologi layan jaring kelas xi tkj 2 smk negeri 1 banyudono :
Pembimbing : Basori, S.Pd., M.Pd

- Judul :upaya tingkat hasil ajar aktif siswa dengan model ajar discovery learning brainstorming ajar jaring dasar kelas x tkj 1 smk negeri 1 banyudono :
Pembimbing : Basori, S.Pd., M.Pd

- Judul :tinjau pengaruh game-based learning kembang computational thinking skills perspektif teori brennan resnick :
Pembimbing : Cucuk Wawan Budiyanto, ST., Ph.D

- Judul :studi evaluasi game edukatif didik sd kelas rendah ajar hitung tinjau aspek gamifikasi kurikulum :
Pembimbing : Endar Suprih W., S.T.,M.Eng
